# Σημειωματάριο τρίτο: Matching the DataFrames

## Όπου ενώνουμε τα tweets με τα CVE-IDs στα οποία αναφέρονται

Έχοντας σχηματίσει τα σύνολα δεδομένων με τα tweets και τις εγγραφές των ευπαθειών, σε αυτό το notebook αντιστοιχίζουμε κάθε tweet με τις ευπάθειες στις οποίες αυτό αναφέρεται. Ο **σκοπός** μας είναι να δημιουργήσουμε ένα dataset με tweets που αναφέρονται σε έγκυρους κωδικούς CVE-IDs, έτσι ώστε να αναλύσουμε σε επόμενα notebooks την δημοτικότητα των ευπαθειών. Είναι ιδιαίτερα σημαντικός ο έλεγχος αυτός γιατί μπορεί ένα tweet (ίσως από κάποιο bοt account) να αναφέρεται σε ένα ID που δεν υπάρχει ως επίσημη εγγραφή στην βάση δεδομένων της NIST. Τέτοια tweets θα διαγραφούν καθώς αποτελούν θόρυβο στα δεδομένα μας και θα αλλοίωναν το αποτέλεσμα της ανάλυσης.

In [1]:
import pandas as pd
import numpy as np
import re 
from ast import literal_eval
import traceback

In [2]:
# Χρησιμοποιούμε την παράμετρο lineterminator γιατί χωρίς αυτήν τα δεδομένα δεν "διαβάζονται" ορθά. 
# Ο λόγος είναι ίσως ότι τα CSV αρχεία δημιουργήθηκαν σε υπολογιστή με λειτουργικό σύστημα Linux.
tweets = pd.read_csv('Data/tweets/tweets_2021_with_users.csv', lineterminator='\n')
records = pd.read_csv('Data/NVD/NVD_records_2022_downoload_at_February_2022_01-02-2022.csv', lineterminator='\n')

#### Ο αλγόριθμος

Στην συνέχεια περιγράφεται ο αλγόριθμος που υλοποιεί το matching. 

Για κάθε ένα από τα tweets εκτελούνται τα ακόλουθα βήματα:
1. Χρησιμοποιώντας regular expression, εντοπίζουμε αρχικά όλα τα CVE IDs που υπάρχουν στο κείμενο του tweet και κρατάμε σε ένα set τα ξεχωριστά IDs που εντοπίσαμε.
2. Στην συνέχεια, για κάθε ένα από αυτά τα IDs που βρήκαμε στο προηγούμενο βήμα, κρατάμε σε μια λίστα τα έγκυρα IDs, δηλαδή τα IDs που υπάρχουν ως εγγραφές στο NVD DataFrame μας το οποίο περιέχει τις εγγραφές με τα επίσημα δεδομένα ευπαθειών από την NVD. 
3. Προσθέτουμε στην λίστα *valid_CVE_IDs_matched* τα έγκυρα CVE IDs που βρήκαμε σε αυτό το tweet. Η λίστα αυτή περιέχει για κάθε tweet μια λίστα με τα έγκυρα CVE IDs που αναφέρονται σε αυτό ή NaN εάν στο συγκεκριμένο tweet δεν αναφέρεται κανένα έγκυρο CVE ID.
4. Προσθέτουμε στην λίστα *valid_CVE_CVSSscores* τα CVSS scores των έγκυρων ευπαθειών που αναφέρονται σε αυτό το tweet.
5. Προσθέτουμε στην λίστα *valid_CVE_CWEs* τα CWEs των έγκυρων ευπαθειών που αναφέρονται σε αυτό το tweet.
6. Προσθέτουμε στην λίστα *valid_CVE_associated_vendors*, τους vendors των προϊόντων που σχετίζονται με τις ευπάθειες που αναφέρονται σε αυτό το tweet.
7. Προσθέτουμε στην λίστα *valid_associated_with_application* τον αριθμό '1' στην περίπτωση που έστω και μια από τις (έγκυρες) ευπάθειες στις οποίες αναφέρεται το tweet, είναι ευπάθεια που επηρεάζει λογισμικό εφαρμογών. Εάν καμιά από τις ευπάθειες αυτές δεν σχετίζεται με λογισμικό εφαρμογών, τότε αυτό σημειώνεται με την προσθήκη ενός '0'.
8. Εντελώς αντίστοιχα προσθέτουμε τον αριθμό '1' στην λίστα *valid_associated_with_hardware* στην περίπτωση που έστω και μια από τις (έγκυρες) ευπάθειες στις οποίες αναφέρεται το tweet, είναι ευπάθεια που επηρεάζει hardware.
9. Ακόμη, προσθέτουμε τον αριθμό '1' στην λίστα *valid_associated_with_os* στην περίπτωση που έστω και μια από τις (έγκυρες) ευπάθειες στις οποίες αναφέρεται το tweet, είναι ευπάθεια που επηρεάζει Operating System.

Όλες οι παραπάνω λίστες στις οποίες αναφερόμαστε είναι παράλληλες λίστες με την λίστα texts, η οποία περιέχει το κείμενο του κάθε tweet.

Είναι κάπως περίπλοκη η διαδικασία ενσωμάτωσης των πληροφοριών των ευπαθειών με τα tweet διότι σε κάθε tweet μπορεί ο συγγραφέας να αναφέρεται σε πάνω από μία ευπάθειες. Εάν σε κάθε tweet σημειώναμε ένα μόνο CVE ID, τότε θα ενώναμε τις πληροφορίες της ευπάθειας με αυτές του tweet χρησιμοποιώντας την συνάρτηση merge() της βιβλιοθήκης pandas.

#### Προεργασία

Πριν από το matching θέλουμε να μετατρέψουμε σε λίστες τα αντικείμενα λιστών που τώρα έχουν αναγνωσθεί ως str.Για να το κάνουμε αυτό θα καλέσουμε την literal_eval(). Επειδή όμως δεν μπορεί να τρέξει εάν έχουμε τιμές NaN, μετατρέπουμε τα NaN σε None τα οποία είναι αποδεκτά από την βιβλιοθήκη. 

Αξίζει να σημειωθεί ότι σε αυτό το σημείο δοκιμάζοντας να εμφανίσουμε γραμμές με NaNs εντοπίσαμε ότι τελικά στο data set υπάρχουν μερικές εγγραφές ** REJECTED \** . Οπότε ενημερώθηκε και το 2o notebook όπου στην προηγούμενη έκδοσή του κάναμε την αναζήτηση χωρίς να συμπεριλάβουμε κενό στην απόσταση ανάμεσα στους δύο αστερίσκους και τη λέξη REJECTED. Με αυτό τον τρόπο δεν βρίσκαμε τέτοιες εγγραφές. Τώρα όμως τοποθετήθηκε το κενό και εντοπίζονται. 

Πηγές που βρέθηκαν για την αντιμετώπιση αυτού του προβλήματος:
- https://stackoverflow.com/questions/23111990/pandas-dataframe-stored-list-as-string-how-to-convert-back-to-list
- https://stackoverflow.com/questions/59352974/how-can-i-convert-nan-1-2-3-string-to-list-nan-1-2-3
- https://www.geeksforgeeks.org/replace-nan-values-with-zeros-in-pandas-dataframe/

In [3]:
def prep(aString):
    '''
    Συνάρτηση που λαμβάνει ως είσοδο μια λίστα σε μορφή string και επιστρέφει την λίστα.
    Λαμβάνεται η φροντίδα ώστε να τρέξει ακόμα και με NaNs.
    Αναφορά: # https://stackoverflow.com/questions/59352974/how-can-i-convert-nan-1-2-3-string-to-list-nan-1-2-3
    '''
    aString = str(aString)
    return literal_eval(aString.replace('nan', 'None'))

In [4]:
records['CWEs'] = records['CWEs'].apply(lambda cwes: prep(cwes))
records['associatedVendors'] = records['associatedVendors'].apply(lambda vendors: prep(vendors))
records['associatedProductsType'] = records['associatedProductsType'].apply(lambda prod_types: prep(prod_types))

In [5]:
# Κρατάμε στην λίστα texts την στήλη με τα κείμενα των tweets, σε κάθε ένα από τα οποία θα κάνουμε αναζήτηση με
# regular expression, και όλες τις σχετικές στήλες του DataFrame records.
texts = list(tweets['text'])  
nvd_ids = list(records['cve_id'])
nvd_cvss = list(records['CVSS_baseScore'])
nvd_cwes = list(records['CWEs'])
nvd_vendors = list(records['associatedVendors'])
nvd_products_type = list(records['associatedProductsType'])

# Βοηθητικές λίστες που χρειαζόμαστε για την συλλογή των δεδομένων που θα μπουν με insert στο dataframe των tweets.
valid_CVE_IDs_matched = []
valid_CVE_CVSSscores = [] 
valid_CVE_CWEs = [] 
valid_CVE_associated_vendors = []
valid_associated_with_application = []
valid_associated_with_hardware = [] 
valid_associated_with_os = [] 

#### Η υλοποίηση του αλγορίθμου

In [6]:
# Χρησιμοποιούμε έναν μετρητή για να εμφανίζουμε ανά 10.000 τα tweets που έχουμε προσπελάσει.
i = 0 

for text in texts:
    if (i % 10000 == 0):
        print("Έως τώρα έχουν προσπελαστεί", i, "tweets.")
    i = i +1   
    
    try:
        
        # Η ids είναι η υπολίστα στην οποία αποθηκεύονται τα CVE IDs που αναφέρονται στο κείμενο
        # του συγκεκριμένου tweet και έχουν ελεγχθεί ότι περιλαμβάνονται στην NVD.
        ids = []
        
        # Η cvss είναι η υπολίστα στην οποία αποθηκεύονται τα CVSS scores των CVE IDs που αναφέρονται στο
        # κείμενο του συγκεκριμένου tweet.
        cvss = []
        
        # Η cwes είναι η υπολίστα στην οποία αποθηκεύονται τα CWEs των CVE IDs που αναφέρονται στο
        # κείμενο του συγκεκριμένου tweet.
        cwes = []
        
        # Η vendors είναι η υπολίστα στην οποία αποθηκεύονται οι vendors των προϊόντων που σχετίζονται με τα
        # CVE IDs που αναφέρονται στο κείμενο του συγκεκριμένου tweet.
        vendors = [] 
        
        # Οι παρακάτω λίστες είναι υπολίστες στις οποίες αποθηκεύουμε για κάθε έγκυρη ευπάθεια που αναφέρεται στο 
        # tweet το αν αυτή σχετίζεται με λογισμικό εφαρμογών, υλικό ή λογισμικό συστήματος αντίστοιχα.
        isApplication = []
        isHardware = []
        isOS = []
        
        # Με την findall βρίσκουμε τα CVE IDs που αναφέρονται στο tweet χρησιμοποιώντας regular expression.
        # Είναι όμως απαραίτητος ο έλεγχος που γίνεται στην συνέχεια για το αν αυτό το ID υπάρχει στην βάση 
        # δεδομένων της NVD έτσι ώστε να μην κρατήσουμε μη έγκυρα CVE IDs (δηλ. IDs που δεν υπάρχουν στο
        # dataset που φτιάξαμε από τα επίσημα δεδομένα της NVD). 
        #
        # Η findall επιστρέφει μια λίστα με τα strings που ταιριάζουν στο pattern που έχουμε δηλώσει, και αυτό σημαίνει
        # ότι επιτρέπει τα διπλότυπα. Έτσι εάν σε ένα tweet έχουμε ένα ID που αναφέρεται δύο φορές, θα αποθηκευτεί
        # στη λίστα δύο φορές. Εμείς όμως θέλουμε από κάθε tweet να κρατάμε τα μοναδικά IDs που αναφέρονται σε αυτό.
        # Οπότε μετατρέπουμε την λίστα σε set κι έτσι διαγράφουμε τα διπλότυπα.
        matched_ids = set(re.findall("CVE-\d{1,6}-\d{1,6}", text))  
        
        for matched in matched_ids:
            
            # Εάν το CVE ID που βρήκε η regular expression υπάρχει ως εγγραφή στην NVD (είναι δηλαδή έγκυρο) τότε 
            # κράτησέ το και μαζί του κράτησε και όλες τις υπόλοιπες πληροφορίες.
            if matched in nvd_ids:
                ids.append(matched)
                
                # Επειδή είναι παράλληλη η λίστα των CVE IDs, με τις υπόλοιπες λίστες όπου έχουμε κρατήσει
                # από τις στήλες του DataFrame records, μπορούμε να κρατήσουμε 
                # τώρα και τις πληροφορίες από τις λίστες αυτές.
                
                # Εντοπισμός του index της γραμμής που έχει τις πληροφορίες στο records ώστε να τις κρατήσουμε.
                index = nvd_ids.index(matched)
                
                # Και τώρα χρησιμοποιώντας αυτό το index, εντοπίζουμε το CVSS score.
                cvss.append(nvd_cvss[index])
                
                # Εντοπίζουμε και αποθηκεύουμε και τα CWEs της κάθε ευπάθειας.
                if nvd_cwes[index] != None:
                    for cwe in nvd_cwes[index]:
                        cwes.append(cwe)
                
                # Εντοπίζουμε και αποθηκεύουμε και τους vendors.
                if nvd_vendors[index] != None:
                    for vendor in nvd_vendors[index]:
                        vendors.append(vendor)
                
                # Και σημειώνουμε με 1 την τιμή του product type για τους 3 δυνατούς διαφορετικούς τύπους.
                if nvd_products_type[index] != None:
                    
                    if 'a' in nvd_products_type[index]:
                        isApplication.append(1)
                    else:
                        isApplication.append(0)
                    
                    if 'h' in nvd_products_type[index]:
                        isHardware.append(1)
                    else:
                        isHardware.append(0)
                    
                    if 'o' in nvd_products_type[index]:
                        isOS.append(1)
                    else:
                        isOS.append(0)

                
        # Εάν δεν βρήκαμε κανένα έγκυρο CVE ID στο κείμενο του text τότε καταχώρησε np.NaN
        # στην θέση της λίστας valid_CVE_IDs_matched που αντιστοιχεί στο tweet αυτό.        
        if len(ids) == 0:
            valid_CVE_IDs_matched.append(np.NaN)     
   
        else:
            # Αποθήκευση των έγκυρων IDs που εντοπίσαμε στο κείμενο.
            valid_CVE_IDs_matched.append(ids)
            
        # Εάν δεν βρήκαμε CVSS score για το CVE ID που αναφέρεται στο κείμενο του text τότε καταχώρησε np.NaN
        # στην θέση της λίστας valid_CVE_CVSSscores που αντιστοιχεί στο tweet αυτό.        
        if len(cvss) == 0:
            valid_CVE_CVSSscores.append(np.NaN)     
   
        else:
            # Αποθήκευση των CVSS scores των έγκυρων IDs που εντοπίσαμε στο κείμενο.
            valid_CVE_CVSSscores.append(cvss)
        
        # Εάν δεν βρήκαμε CWEs για το CVE ID που αναφέρεται στο κείμενο του text τότε καταχώρησε np.NaN
        # στην θέση της λίστας valid_CVE_CWEs που αντιστοιχεί στο tweet αυτό.        
        if len(cwes) == 0:
            valid_CVE_CWEs.append(np.NaN)     
   
        else:
            # Αποθήκευση των CWEs των έγκυρων IDs που εντοπίσαμε στο κείμενο.
            valid_CVE_CWEs.append(cwes)
        
        # Εάν δεν βρήκαμε vendors για το CVE ID που αναφέρεται στο κείμενο του text τότε καταχώρησε np.NaN
        # στην θέση της λίστας valid_CVE_associated_vendors που αντιστοιχεί στο tweet αυτό.        
        if len(vendors) == 0:
            valid_CVE_associated_vendors.append(np.NaN)     
   
        else:
            # Αποθήκευση των vendors των έγκυρων IDs που εντοπίσαμε στο κείμενο.
            valid_CVE_associated_vendors.append(vendors)
        
        # Εάν τουλάχιστον μία από τις ευπάθειες στις οποίες αναφέρεται το tweet είναι ευπάθειες που επηρεάζουν 
        # λογισμικό εφαρμογών, τότε σημείωσε ότι ναι, το tweet αυτό σχετίζεται με λογισμικό εφαρμογών.
        if 1 in isApplication:
            valid_associated_with_application.append(1)
        else:
            valid_associated_with_application.append(0)
            
        # Παρόμοια και για το Hardware.
        if 1 in isHardware:
            valid_associated_with_hardware.append(1)
        else:
            valid_associated_with_hardware.append(0)
            
        # Παρόμοια και για το OS.
        if 1 in isOS:
            valid_associated_with_os.append(1)
        else:
            valid_associated_with_os.append(0)
        
    except Exception as e:
        tb = traceback.format_exc()
        print(tb)
        print("Ένα σφάλμα εμφανίστηκε:")
        print(e)
        break

Έως τώρα έχουν προσπελαστεί 0 tweets.
Έως τώρα έχουν προσπελαστεί 10000 tweets.
Έως τώρα έχουν προσπελαστεί 20000 tweets.
Έως τώρα έχουν προσπελαστεί 30000 tweets.
Έως τώρα έχουν προσπελαστεί 40000 tweets.
Έως τώρα έχουν προσπελαστεί 50000 tweets.
Έως τώρα έχουν προσπελαστεί 60000 tweets.
Έως τώρα έχουν προσπελαστεί 70000 tweets.
Έως τώρα έχουν προσπελαστεί 80000 tweets.
Έως τώρα έχουν προσπελαστεί 90000 tweets.
Έως τώρα έχουν προσπελαστεί 100000 tweets.
Έως τώρα έχουν προσπελαστεί 110000 tweets.
Έως τώρα έχουν προσπελαστεί 120000 tweets.
Έως τώρα έχουν προσπελαστεί 130000 tweets.
Έως τώρα έχουν προσπελαστεί 140000 tweets.
Έως τώρα έχουν προσπελαστεί 150000 tweets.
Έως τώρα έχουν προσπελαστεί 160000 tweets.
Έως τώρα έχουν προσπελαστεί 170000 tweets.
Έως τώρα έχουν προσπελαστεί 180000 tweets.
Έως τώρα έχουν προσπελαστεί 190000 tweets.
Έως τώρα έχουν προσπελαστεί 200000 tweets.
Έως τώρα έχουν προσπελαστεί 210000 tweets.


Εφόσον η αντιστοίχηση των tweets με τα CVE IDs ολοκληρώθηκε επιτυχώς, εισάγουμε τις λίστες τις οποίες γεμίσαμε με δεδομένα από τις αντιστοιχίες, σαν νέες στήλες στο DataFrame **tweets**.

In [7]:
if len(tweets) == len(valid_CVE_IDs_matched) == len(valid_CVE_CVSSscores) == len(valid_CVE_CWEs) == len(valid_CVE_associated_vendors) == len(valid_associated_with_application) == len(valid_associated_with_hardware) == len(valid_associated_with_os):
    print("Το matching ολοκληρώθηκε επιτυχώς! Οι λίστες προστίθενται σαν νέα στήλες στο DataFrame tweets.")
    tweets.insert(2, 'matched_CVE_IDs', valid_CVE_IDs_matched)
    tweets.insert(3, 'CVSS_of_CVE_IDs', valid_CVE_CVSSscores)
    tweets.insert(4, 'CWEs_of_CVE_IDs', valid_CVE_CWEs)
    tweets.insert(5, 'assosiated_vendors_of_CVE_IDs', valid_CVE_associated_vendors)
    tweets.insert(6, 'tweet_refers_to_application_product?', valid_associated_with_application)
    tweets.insert(7, 'tweet_refers_to_hardware_product?', valid_associated_with_hardware)
    tweets.insert(8, 'tweet_refers_to_os_product?', valid_associated_with_os)

else:
    print("Κάτι πήγε στραβά και το μέγεθος της λίστας data δεν είναι ίσο με\n το μέγεθος του dataframe tweets οπότε δεν μπορούμε να τα ενώσουμε...Ξαναδές το!")

Το matching ολοκληρώθηκε επιτυχώς! Οι λίστες προστίθενται σαν νέα στήλες στο DataFrame tweets.


In [8]:
tweets

,id,text,matched_CVE_IDs,CVSS_of_CVE_IDs,CWEs_of_CVE_IDs,assosiated_vendors_of_CVE_IDs,tweet_refers_to_application_product?,tweet_refers_to_hardware_product?,tweet_refers_to_os_product?,created_at,...,like_count,quote_count,author_id,author_name,author_username,author_description,author_followers_count,author_following_count,author_tweet_count,author_listed_count
0,1458428897368940553,CVE-2021-34582\n\nIn Phoenix Contact FL MGUARD...,[CVE-2021-34582],[4.8],[CWE-79],[phoenixcontact],0,0,1,2021-11-10T13:38:40.000Z,...,0,0,941389496771399680,Vulmon Vulnerability Feed,VulmonFeeds,Vulnerability Feed Bot (tweets new and some ol...,1830,2,85176,46
1,1458428424326983680,SAP объявила об исправлении критических уязвим...,NaN,NaN,NaN,NaN,0,0,0,2021-11-10T13:36:47.000Z,...,0,0,1132127578922344448,IT news for all,IT_news_for_all,Сбор новостей с каналов ИТ тематики\n🔥🎯https:/...,20,17,4685,1
2,1458428142373179392,CVE-2021-34598\n\nIn Phoenix Contact FL MGUARD...,[CVE-2021-34598],[7.5],[CWE-401],[phoenixcontact],0,0,1,2021-11-10T13:35:40.000Z,...,0,0,941389496771399680,Vulmon Vulnerability Feed,VulmonFeeds,Vulnerability Feed Bot (tweets new and some ol...,1830,2,85176,46
3,1458427987301371911,New post from https://t.co/uXvPWJy6tj (CVE-202...,[CVE-2021-39474],[7.2],[CWE-78],[ubeeinteractive],0,0,1,2021-11-10T13:35:03.000Z,...,0,0,955014888446939136,Wolfgang Sesin,WolfgangSesin,"Check Point Master & Instructor, Pentest Exper...",292,494,203762,9
4,1458427985753776138,New post from https://t.co/9KYxtdZjkl (CVE-202...,[CVE-2021-39474],[7.2],[CWE-78],[ubeeinteractive],0,0,1,2021-11-10T13:35:02.000Z,...,0,0,958005194398289920,www.sesin.at,www_sesin_at,for more information about us please visit htt...,88,0,212995,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210094,1346143661831172097,CVE-2021-26287 is called Confident Sickle\nhtt...,NaN,NaN,NaN,NaN,0,0,0,2021-01-04T17:17:13.000Z,...,0,0,1096056138569842688,vulnonym,vulnonym,I'm a bot generating names for CVE IDs.,902,0,37553,16
210095,1346143642998743043,CVE-2021-26286 shall henceforth be named Relia...,NaN,NaN,NaN,NaN,0,0,0,2021-01-04T17:17:09.000Z,...,0,0,1096056138569842688,vulnonym,vulnonym,I'm a bot generating names for CVE IDs.,902,0,37553,16
210096,1346143619967811586,"One night, CVE-2021-25847 wished upon a star, ...",[CVE-2021-25847],[9.1],[CWE-125],[moxa],0,0,1,2021-01-04T17:17:03.000Z,...,0,0,1096056138569842688,vulnonym,vulnonym,I'm a bot generating names for CVE IDs.,902,0,37553,16
210097,1346143576116375555,Let the annals of the day show that CVE-2021-1...,NaN,NaN,NaN,NaN,0,0,0,2021-01-04T17:16:53.000Z,...,0,0,1096056138569842688,vulnonym,vulnonym,I'm a bot generating names for CVE IDs.,902,0,37553,16


Θα πρέπει στη συνέχεια να διαχειριστούμε τα tweets που δεν αναφέρονται σε έγκυρα CVE IDs.

In [9]:
if (valid_CVE_IDs_matched.count(np.NaN) == 0):
    print("Τέλεια, κάθε tweet στο dataset αναφέρεται σε τουλάχιστον ένα έγκυρο CVE ID!")
else:
    print("Έχουμε ",valid_CVE_IDs_matched.count(np.NaN), " tweets τα οποία δεν αναφέρονται σε κάποιο έγκυρο CVE ID.")

Έχουμε  10484  tweets τα οποία δεν αναφέρονται σε κάποιο έγκυρο CVE ID.


In [10]:
tweets.isna().sum()

id                                          0
text                                        0
matched_CVE_IDs                         10484
CVSS_of_CVE_IDs                         10484
CWEs_of_CVE_IDs                         11091
assosiated_vendors_of_CVE_IDs           11096
tweet_refers_to_application_product?        0
tweet_refers_to_hardware_product?           0
tweet_refers_to_os_product?                 0
created_at                                  0
retweet_count                               0
reply_count                                 0
like_count                                  0
quote_count                                 0
author_id                                   0
author_name                                 6
author_username                             0
author_description                       3863
author_followers_count                      0
author_following_count                      0
author_tweet_count                          0
author_listed_count               

Παρατηρούμε ότι 10.484 tweets από το σύνολο των 210.099 δεν αναφέρονται σε κάποιο έγκυρο CVE ID.
Θα πρέπει να διαγραφούν από το dataset πριν προχωρήσουμε στην ανάλυση των δεδομένων.

In [11]:
tweets = tweets.dropna(subset=['matched_CVE_IDs'])

In [12]:
tweets.isna().sum()

id                                         0
text                                       0
matched_CVE_IDs                            0
CVSS_of_CVE_IDs                            0
CWEs_of_CVE_IDs                          607
assosiated_vendors_of_CVE_IDs            612
tweet_refers_to_application_product?       0
tweet_refers_to_hardware_product?          0
tweet_refers_to_os_product?                0
created_at                                 0
retweet_count                              0
reply_count                                0
like_count                                 0
quote_count                                0
author_id                                  0
author_name                                5
author_username                            0
author_description                      3182
author_followers_count                     0
author_following_count                     0
author_tweet_count                         0
author_listed_count                        0
dtype: int

Τα tweets που δεν αναφέρονται σε έγκυρο CVE ID έχουν πια διαγραφεί. Παρατηρούμε ότι έχουμε missing values στις στήλες όπου αποθηκεύουμε vendors και  CWEs και είναι αναμενόμενο μιας και κάποιες εγγραφές στην NVD δεν έχουν τις πληροφορίες αυτές (τουλάχιστον δεν τις είχαν την στιγμή και την μέρα που έγινε η λήψη των δεδομένων).

Τα missing values στην στήλη 'author_description' ήταν επίσης αναμενόμενα. Όχι όμως αυτά της στήλης 'author_name', αφού στο πρώτο notebook όπου συλλέγουμε τα tweets δεν φαίνονταν να υπάρχουν ελλιπείς τιμές σε αυτό το χαρακτηριστικό πριν δημιουργήσουμε το αρχείο CSV. Ας δούμε τί βρίσκεται εκεί...

In [13]:
tweets.loc[tweets['author_name'].isna()]

,id,text,matched_CVE_IDs,CVSS_of_CVE_IDs,CWEs_of_CVE_IDs,assosiated_vendors_of_CVE_IDs,tweet_refers_to_application_product?,tweet_refers_to_hardware_product?,tweet_refers_to_os_product?,created_at,...,like_count,quote_count,author_id,author_name,author_username,author_description,author_followers_count,author_following_count,author_tweet_count,author_listed_count
4748,1456644114124926979,A heap overflow #vulnerability (CVE-2021-43267...,[CVE-2021-43267],[9.8],[CWE-20],"[netapp, linux, fedoraproject]",0,0,1,2021-11-05T15:26:34.000Z,...,5,0,717369822007275520,NaN,rul3r,NaN,1798,0,4691,111
8027,1455595683994476548,WARNING: A critical unauthenticated remote cod...,[CVE-2021-22205],[10.0],[CWE-20],[gitlab],1,0,0,2021-11-02T18:00:29.000Z,...,2,0,717369822007275520,NaN,rul3r,NaN,1798,0,4691,111
71760,1428376802393792516,FireEye has disclosed a new critical #vulnerab...,[CVE-2021-28372],[8.3],[CWE-290],[throughtek],1,0,0,2021-08-19T15:22:22.000Z,...,2,1,717369822007275520,NaN,rul3r,NaN,1798,0,4691,111
105764,1412421816400986122,#Microsoft is urging #Azure users to update th...,[CVE-2021-26701],[9.8],[NVD-CWE-noinfo],"[fedoraproject, microsoft]",1,0,1,2021-07-06T14:42:57.000Z,...,2,0,717369822007275520,NaN,rul3r,NaN,1798,0,4691,111
173827,1374023488617336842,A critical #vulnerability (CVE-2021-22986 / CV...,[CVE-2021-22986],[9.8],[NVD-CWE-noinfo],[f5],1,0,0,2021-03-22T15:41:42.000Z,...,1,0,717369822007275520,NaN,rul3r,NaN,1798,0,4691,111


Συμπαιραίνουμε ότι εδώ η βιβλιοθήκη pandas θεωρεί missing values τις τιμές της στήλη 'author_name' για έναν συγκεκριμένο λογαριασμό επειδή έχει για όνομα το null. Θα το παραβλέψουμε όμως γιατί είναι ένα όνομα που δοθεί στον λογαριασμό αυτόν από τον [κάτοχό του](https://twitter.com/rul3r), οπότε δεν είναι ορθό να θεωρηθεί ως ελλιπής τιμή. Μέχρι την στιγμή της συγγραφής αυτού του notebook δεν βρέθηκε τρόπος να κάνουμε την isna() της pandas να παραβλέψει αυτό το χαρακτηριστικό, οπότε απλά θα το έχουμε υπόψη μας στη συνέχεια...

Οπότε ολοκληρώνεται εδώ ο σκοπός αυτού του notebook με την αποθήκευση του νέου DataFrame σε ένα αρχείο CSV.

In [14]:
tweets.to_csv('Data/tweets/tweets_2021_with_users_matched_joined.csv', index=False)